In [1]:
import os
import pandas as pd

os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks')
import multimodal_training as mt

0
Loading existing mappings from /home/jcsanguino10/local_citation_model/data/processed


In [21]:
from itertools import chain, combinations

modalities = {
    'bpr': ('item_bpr_embedding', 'user_bpr_embedding'),
    'graph': ('item_graph_embedding', 'user_graph_embedding'),
    'text': ('item_text_embedding', 'user_text_embedding'),
}

sequence = 'user_sequence_embedding'  # Solo usuario

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

final_combinations = []

for combo in powerset(modalities.keys()):
    item_embeds = [modalities[m][0] for m in combo]
    user_embeds = [modalities[m][1] for m in combo]
    final_combinations.append((item_embeds, user_embeds))

with_sequence = [ (items, users + [sequence]) for (items, users) in final_combinations ]
final_combinations.extend(with_sequence)


# Mostrar resultados
for combo in final_combinations:
    print(combo)

print("Total de combinaciones:", len(final_combinations))

(['item_bpr_embedding'], ['user_bpr_embedding'])
(['item_graph_embedding'], ['user_graph_embedding'])
(['item_text_embedding'], ['user_text_embedding'])
(['item_bpr_embedding', 'item_graph_embedding'], ['user_bpr_embedding', 'user_graph_embedding'])
(['item_bpr_embedding', 'item_text_embedding'], ['user_bpr_embedding', 'user_text_embedding'])
(['item_graph_embedding', 'item_text_embedding'], ['user_graph_embedding', 'user_text_embedding'])
(['item_bpr_embedding', 'item_graph_embedding', 'item_text_embedding'], ['user_bpr_embedding', 'user_graph_embedding', 'user_text_embedding'])
(['item_bpr_embedding'], ['user_bpr_embedding', 'user_sequence_embedding'])
(['item_graph_embedding'], ['user_graph_embedding', 'user_sequence_embedding'])
(['item_text_embedding'], ['user_text_embedding', 'user_sequence_embedding'])
(['item_bpr_embedding', 'item_graph_embedding'], ['user_bpr_embedding', 'user_graph_embedding', 'user_sequence_embedding'])
(['item_bpr_embedding', 'item_text_embedding'], ['user_

In [4]:
course_column_combinations

[('item_bpr_embedding',),
 ('user_bpr_embedding',),
 ('item_graph_embedding',),
 ('user_graph_embedding',),
 ('item_text_embedding',),
 ('user_text_embedding',),
 ('item_bpr_embedding', 'user_bpr_embedding'),
 ('item_bpr_embedding', 'item_graph_embedding'),
 ('item_bpr_embedding', 'user_graph_embedding'),
 ('item_bpr_embedding', 'item_text_embedding'),
 ('item_bpr_embedding', 'user_text_embedding'),
 ('user_bpr_embedding', 'item_graph_embedding'),
 ('user_bpr_embedding', 'user_graph_embedding'),
 ('user_bpr_embedding', 'item_text_embedding'),
 ('user_bpr_embedding', 'user_text_embedding'),
 ('item_graph_embedding', 'user_graph_embedding'),
 ('item_graph_embedding', 'item_text_embedding'),
 ('item_graph_embedding', 'user_text_embedding'),
 ('user_graph_embedding', 'item_text_embedding'),
 ('user_graph_embedding', 'user_text_embedding'),
 ('item_text_embedding', 'user_text_embedding'),
 ('item_bpr_embedding', 'user_bpr_embedding', 'item_graph_embedding'),
 ('item_bpr_embedding', 'user_bp

In [2]:
PATH_TO_DATASETS = '/home/jcsanguino10/local_citation_model/data/'

df = pd.read_pickle(f'{PATH_TO_DATASETS}train_binary_all_vectors_128_01_transe_seqvec.pkl')
df_test = pd.read_pickle(f'{PATH_TO_DATASETS}test_binary_all_vectors_128_01_transe_seqvec.pkl')

In [3]:
df.head(1)

,user_id,item_id,item_seq,item_text_embedding,user_text_embedding,user_cum_text_embedding,item_bpr_embedding,user_bpr_embedding,item_graph_embedding,user_graph_embedding,label,full_item_seq,parsed_item_seq,user_sequence_embedding
0,0,6863,[6863],"[2.2087095e-05, 0.01357965, 0.013899612, -0.03...","[-0.022005824, -0.033391304, -0.013403211, -0....","[2.2087095e-05, 0.01357965, 0.013899612, -0.03...","[-0.06554511, 0.082159385, -0.024771133, -0.08...","[-0.16485311, 0.13305487, -0.109800704, -0.159...","[0.027043026, -0.028548103, -0.005808171, -0.1...","[-0.21868221, 0.107340455, 0.20054282, 0.15184...",1,"[6863, 6864]","[2, 3]","[-3.2515903, -4.4112086, 1.4216669, -2.70002, ..."


In [ ]:
results_df = mt.test_multimodal_model(
    columns_to_concat_courses=['item_bpr_embedding', 'item_graph_embedding', 'item_text_embedding'],
    columns_to_concat_users=['user_bpr_embedding', 'user_graph_embedding', 'user_text_embedding', 'user_sequence_embedding'],
    encoding_dims=[720, 512],
    shared_dimensions=128,
    layers_per_modality=[256, 128],
    regularization_bpr_values=[0.005]
)

/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:209: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  course_tensor = [torch.tensor(x, dtype=torch.float) for x in df_binary_temp['course_full_embeddings'].values]
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:212: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_tensor = [torch.tensor(x, dtype=torch.float) for x in df_binary_temp['user_full_embeddings'].values]


Data split: 31456 training samples, 7864 validation samples
Training autoencoder for 100 epochs...
 New best model saved at epoch 1 with val_loss: 0.002803
Epoch 1/100, Train Loss: 0.003223, Val Loss: 0.002803, Best Val Loss: 0.002803
 New best model saved at epoch 2 with val_loss: 0.002576
 New best model saved at epoch 3 with val_loss: 0.002516
 New best model saved at epoch 4 with val_loss: 0.002445
 New best model saved at epoch 5 with val_loss: 0.002414
Epoch 5/100, Train Loss: 0.002485, Val Loss: 0.002414, Best Val Loss: 0.002414
 New best model saved at epoch 6 with val_loss: 0.002405
 New best model saved at epoch 7 with val_loss: 0.002381
 New best model saved at epoch 8 with val_loss: 0.002332
 New best model saved at epoch 9 with val_loss: 0.002274
 New best model saved at epoch 10 with val_loss: 0.002215
Epoch 10/100, Train Loss: 0.002306, Val Loss: 0.002215, Best Val Loss: 0.002215
 New best model saved at epoch 11 with val_loss: 0.002153
 New best model saved at epoch 12 

/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:177: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(pos_course_feat, dtype=torch.float),
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:178: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than tor

 New best model saved at epoch 1 with val_loss: 0.433889
Epoch 1/30, Train Loss: 0.433889, Best Loss: 0.433889
 New best model saved at epoch 2 with val_loss: 0.425346
Epoch 2/30, Train Loss: 0.425346, Best Loss: 0.425346
 New best model saved at epoch 3 with val_loss: 0.402032
Epoch 3/30, Train Loss: 0.402032, Best Loss: 0.402032
 New best model saved at epoch 4 with val_loss: 0.398823
Epoch 4/30, Train Loss: 0.398823, Best Loss: 0.398823
 New best model saved at epoch 5 with val_loss: 0.390082
Epoch 5/30, Train Loss: 0.390082, Best Loss: 0.390082
 New best model saved at epoch 6 with val_loss: 0.385868
Epoch 6/30, Train Loss: 0.385868, Best Loss: 0.385868
 New best model saved at epoch 7 with val_loss: 0.384647
Epoch 7/30, Train Loss: 0.384647, Best Loss: 0.384647
 No improvement for 1/5 epochs
Epoch 8/30, Train Loss: 0.410539, Best Loss: 0.384647
 No improvement for 2/5 epochs
Epoch 9/30, Train Loss: 0.403418, Best Loss: 0.384647
 No improvement for 3/5 epochs
Epoch 10/30, Train Los

Generating recommendations: 100%|██████████| 108/108 [00:00<00:00, 186.18it/s]
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:177: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(pos_course_feat, dtype=torch.float),
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:178: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach()

...

Generating recommendations: 100%|██████████| 108/108 [00:08<00:00, 13.37it/s]
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:194: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(course_feat, dtype=torch.float),


Testing model with use_bpr=binary and fusion_method=by_autoencoder
Using autoencoder fusion method with encoding dimension 512
User feature layer input dim: 512
Course feature layer input dim: 512
Training multimodal model for 30 epochs...
No validation dataset provided - using training loss for model selection
Early stopping: patience=5, delta=1e-05
 New best model saved at epoch 1 with val_loss: 0.482399
Epoch 1/30, Train Loss: 0.482399, Best Loss: 0.482399
 New best model saved at epoch 2 with val_loss: 0.458383
Epoch 2/30, Train Loss: 0.458383, Best Loss: 0.458383
 New best model saved at epoch 3 with val_loss: 0.455846
Epoch 3/30, Train Loss: 0.455846, Best Loss: 0.455846
 New best model saved at epoch 4 with val_loss: 0.451627
Epoch 4/30, Train Loss: 0.451627, Best Loss: 0.451627
 New best model saved at epoch 5 with val_loss: 0.451206
Epoch 5/30, Train Loss: 0.451206, Best Loss: 0.451206
 New best model saved at epoch 6 with val_loss: 0.449574
Epoch 6/30, Train Loss: 0.449574, B

Generating recommendations: 100%|██████████| 108/108 [00:07<00:00, 13.86it/s]
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'user': torch.tensor(user_feat, dtype=torch.float),
/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/notebooks/multimodal_training.py:194: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'course_positive': torch.tensor(course_feat, dtype=torch.float),


Testing model with use_bpr=binary and fusion_method=by_autoencoder
Using autoencoder fusion method with encoding dimension 512
User feature layer input dim: 512
Course feature layer input dim: 512
Training multimodal model for 30 epochs...
No validation dataset provided - using training loss for model selection
Early stopping: patience=5, delta=1e-05
 New best model saved at epoch 1 with val_loss: 0.484073
Epoch 1/30, Train Loss: 0.484073, Best Loss: 0.484073
 New best model saved at epoch 2 with val_loss: 0.458345
Epoch 2/30, Train Loss: 0.458345, Best Loss: 0.458345
 New best model saved at epoch 3 with val_loss: 0.455479
Epoch 3/30, Train Loss: 0.455479, Best Loss: 0.455479
 New best model saved at epoch 4 with val_loss: 0.450547
Epoch 4/30, Train Loss: 0.450547, Best Loss: 0.450547
 New best model saved at epoch 5 with val_loss: 0.449052
Epoch 5/30, Train Loss: 0.449052, Best Loss: 0.449052
 New best model saved at epoch 6 with val_loss: 0.448356
Epoch 6/30, Train Loss: 0.448356, B

Generating recommendations: 100%|██████████| 108/108 [00:07<00:00, 13.89it/s]


In [5]:
results_df

,model_path,use_bpr,fusion_method,reg_lambda,avg_mrr,avg_ndcg_at_k,avg_precision_at_k,avg_custom_precision_at_k
0,/home/jcsanguino10/local_citation_model/models...,bpr,concat,0.005,0.046595,0.064016,0.023838,0.069178
1,/home/jcsanguino10/local_citation_model/models...,bpr,by_autoencoder,0.005,0.106455,0.144567,0.054670,0.160302
2,/home/jcsanguino10/local_citation_model/models...,binary,concat,0.000,0.157383,0.211662,0.076759,0.229693
3,/home/jcsanguino10/local_citation_model/models...,binary,concat,0.000,0.160707,0.215638,0.078217,0.233571
4,/home/jcsanguino10/local_citation_model/models...,binary,by_autoencoder,0.000,0.154318,0.211680,0.079528,0.235723
5,/home/jcsanguino10/local_citation_model/models...,binary,by_autoencoder,0.000,0.151039,0.210068,0.080256,0.238844


In [7]:
import numpy as np


df_test= df_test[['user_id', 'full_item_seq']].drop_duplicates(subset=['user_id'])
temp_df_1 = df[['user_id', 'user_bpr_embedding', 'full_item_seq']].drop_duplicates(subset=['user_id'])
temp_df_2 = df[['item_id', 'item_bpr_embedding']].drop_duplicates(subset=['item_id'])

#create a list of tuples with item_id and the corresponding embedding as a numpy array
item_embeddings = [(row['item_id'], np.array(row['item_bpr_embedding'])) for index, row in temp_df_2.iterrows()]

#create a list of tuples with user_id and the corresponding embedding as a numpy array
user_embeddings = [(row['user_id'], np.array(row['user_bpr_embedding'])) for index, row in temp_df_1.iterrows()]

#create a dictionary where for each user_id we have the item_id that have the best dot product with the user embedding
user_recommendations = {}
for user_id, user_emb in user_embeddings:
    #calculate the dot product between the user embedding and all item embeddings
    dot_products = [(item_id, np.dot(user_emb, item_emb)) for item_id, item_emb in item_embeddings]
    #sort the dot products in descending order
    dot_products = sorted(dot_products, key=lambda x: x[1], reverse=True)
    #get the top 5 item ids checking that they are not in the user's full_item_seq
    top_5_items = [item_id for item_id, _ in dot_products if item_id not in temp_df_1[temp_df_1['user_id'] == user_id]['full_item_seq'].values[0]][:5]
    user_recommendations[user_id] = top_5_items

#Save the recommendations as a new column in the dataframe
df_test['top_5_recommendations'] = df_test['user_id'].map(user_recommendations)

In [8]:
df_test

,user_id,full_item_seq,top_5_recommendations
0,0,[6897],"[6876, 6871, 6870, 6895, 6875]"
1,1,[6920],"[6870, 6871, 6872, 6902, 6876]"
2,2,"[6937, 6961, 6895, 6950, 6964, 6965, 6994, 702...","[6902, 6895, 6894, 6906, 6899]"
16,3,[6996],"[6865, 6899, 6902, 6900, 6906]"
17,4,"[6901, 6865, 6912]","[6871, 6876, 6875, 6902, 6865]"
...,...,...,...
11915,6858,"[6902, 6872]","[6872, 6902, 6870, 6871, 6906]"
11917,6859,"[6865, 6902]","[6865, 6902, 6899, 6906, 6893]"
11919,6860,[6887],"[6920, 6887, 6958, 6864, 6985]"
11920,6861,"[7000, 6901, 6943]","[6870, 6899, 6871, 6906, 6896]"


In [9]:
os.chdir('/home/jcsanguino10/local_citation_model/Secuencial SR')
from evaluation_metrics import calculate_average_mrr, calculate_average_precision_at_k, calculate_average_ndcg_at_k, calculate_average_custom_precision_at_k

k=5

courses_test_dataset = df_test["full_item_seq"].to_list()
courses_recommended_list = df_test["top_5_recommendations"].to_list()

avg_mrr = calculate_average_mrr(courses_test_dataset, courses_recommended_list)
avg_ndcg_at_k = calculate_average_ndcg_at_k(courses_test_dataset, courses_recommended_list, k)
avg_precision_at_k = calculate_average_precision_at_k(courses_test_dataset, courses_recommended_list, k)
avg_custom_precision_at_k = calculate_average_custom_precision_at_k(courses_test_dataset, courses_recommended_list, k)

In [10]:
print(f"MRR: {avg_mrr:.6f}, NDCG@{k}: {avg_ndcg_at_k:.6f}, Precision@{k}: {avg_precision_at_k:.6f}, Custom Precision@{k}: {avg_custom_precision_at_k:.6f}")

MRR: 0.207749, NDCG@5: 0.277581, Precision@5: 0.100219, Custom Precision@5: 0.308478
